In [ ]:
# Example
of the last added functionalities in onlineSG

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [1]:
import sys
sys.path.append('../')  # uncomment to add the path to the onlineSG folder 
# You have to change the path if you move this file.
# Alternatevely add the onlineSG folder to your PYTHONPATH
# e.g. On Linux add this line on .bashrc:
# export PYTHONPATH="<absolute-path-to-onlineSG>:PYTHONPATH"
# then you can use onlineSG library from everywhere

In [31]:
import source.game as game
import source.player as player
import source.environment as environment
import source.errors as errors
import source.parsers as parsers
import source.players.attackers as attackers
import source.players.base_defenders as base_defenders
import source.players.defenders as defenders
import source.runner as runner
import source.players.detection as detection 

from source.util import *
import numpy as np
import pandas as pd
from importlib import *
from source.util import *
import time

In [ ]:
print("ciao")

In [20]:
#c = runner.Configuration(g, "detector", print_results=False)

In [23]:
# if you change something in the files, don't forget to reload
reload(player)
reload(base_defenders)
reload(attackers)
reload(defenders)
reload(errors)
reload(runner)
reload(parsers)
reload(game)
#reload(environment)
reload(detection)

<module 'source.players.detection' from '/home/lorenzo/Scrivania/Polimi/Thesis/code/onlineSG/source/players/detection.py'>

In [35]:
time_horizon = 100
T = 4
targets = (5,4,8,1)
values = tuple((v, v) for v in targets)
distribution1 = (0.17,0.8, 0.01, 0.02)
g = game.Game(values, time_horizon)
agent = detection.HOLMES(g, 0, 1, 1)
#profiles = [attackers.StackelbergAttacker(g, 1, 1)] #,
profiles = [attackers.StochasticAttacker(g, 1, 1, *distribution1)]#, attackers.StochasticAttacker(g, 1, 1, *distribution2)] #,
attacker = attackers.StackelbergAttacker(g, 1, 1)
#attacker = attackers.StochasticAttacker(g, 1, 1, *distribution1)
g.set_players([agent], [attacker], profiles)

In [36]:
e = runner.Experiment(g)

In [32]:
start_time = time.time()
e.run_interaction()
print(game_str(e.game, lenght=7))
print(e.agent.belief)
print(e.game.strategy_history[-1])
print(time.time() - start_time)

□	 	 	 	
5	4	8	1	
 	△	 	 	
----------------------------
 	□	 	 	
5	4	8	1	
 	△	 	 	
{<StochasticAttacker id:1 resources:1>: 1.0, <StackelbergAttacker id:1 resources:1>: 0.0}
{0: (0.30434782608695643, 0.13043478260869565, 0.5652173913043479, 0.0), 1: [0, 1, 0, 0]}
2.4693214893341064


In [37]:
start_time = time.time()
e.run()
print(time.time() - start_time)

378.01570677757263


In [ ]:
start_time = time.time()

In [24]:
time_horizon = 100
targets = (1,2,3)
values = tuple((v, v) for v in targets)
distribution1 = (0,0.5,0.5)
distribution2 = (0,0.1,0.9)
g = game.Game(values, time_horizon)
#profiles = [attackers.StackelbergAttacker(g, 1, 1)] #,
profiles = [attackers.StochasticAttacker(g, 1, 1, *distribution1)]#, attackers.StochasticAttacker(g, 1, 1, *distribution2)] #,
agent = detection.StrategyAwareDetector(g, 0, 1)
attacker = attackers.StackelbergAttacker(g, 1, 1)
#attacker = attackers.StochasticAttacker(g, 1, 1, *distribution1)
g.set_players([agent], [attacker], profiles)

In [26]:
c = runner.Configuration(g, "concurrent", "prova", False)

In [25]:
rm -rf concurrent

In [27]:
c.run(3)

In [724]:
e = runner.Experiment(g)

In [756]:
e.run_interaction()
#print(game_str(e.game, lenght=7))
print(e.agent.belief, e.agent.detection)
print(e.game.strategy_history[-1])

[0, 0.9, 0.1]
0.0
False
0.5
False
0.5
False
{<StochasticAttacker id:1 resources:1>: 1.853020188851843e-17, <StackelbergAttacker id:1 resources:1>: 1.0} None
{0: [0.0, 0.39, 0.61], 1: [0, 1, 0]}


In [757]:
e.agent.tau

32

In [720]:
e.run()

[0, 0.01, 0.99]
0.0
False
1.0
False
0.0
False
[0, 0.01, 0.99]
0.0
False
1.0
False
0.0
False
[0, 0.01, 0.99]
0.0
False
1.0
False
0.0
False
[0, 0.01, 0.99]
0.0
False
1.0
False
0.0
False
[0, 0.01, 0.99]
0.0
False
1.0
False
0.0
False
[0, 0.01, 0.99]
0.0
False
1.0
False
0.0
False
[0, 0.01, 0.99]
0.0
False
1.0
False
0.0
False
[0, 0.01, 0.99]
0.0
False
1.0
False
0.0
False
[0, 0.01, 0.99]
0.0
False
1.0
False
0.0
False
[0, 0.01, 0.99]
0.0
False
1.0
False
0.0
False
[0, 0.01, 0.99]
0.0
False
1.0
False
0.0
False
[0, 0.01, 0.99]
0.0
False
1.0
False
0.0
False
[0, 0.01, 0.99]
0.0
False
1.0
False
0.0
False
[0, 0.01, 0.99]
0.0
False
1.0
False
0.0
False
[0, 0.01, 0.99]
0.0
False
1.0
False
0.0
False
[0, 0.01, 0.99]
0.0
False
1.0
False
0.0
False
[0, 0.01, 0.99]
0.0
False
1.0
False
0.0
False
[0, 0.01, 0.99]
0.0
False
1.0
False
0.0
False
[0, 0.01, 0.99]
0.0
False
1.0
False
0.0
False
[0, 0.01, 0.99]
0.0
False
1.0
False
0.0
False
[0, 0.01, 0.99]
0.0
False
1.0
False
0.0
False
[0, 0.01, 0.99]
0.0
False
1.0
Fals

In [74]:
time_horizon = 100
T = 4
targets = (5,4,8,1)
values = tuple((v, v) for v in targets)
distribution1 = (0.17,0.8, 0.01, 0.02)
g = game.Game(values, time_horizon)

other_kinds = [attackers.StochasticAttacker(g, 1, 1, *distribution1)] #,
strategy_aware = attackers.StackelbergAttacker(g, 1, 1)
L = 2
agent = detection.StrategyAwareDetector.parse("sad1-0.01@stackelberg@stochastic_attacker1-0.17-0.8-0.01-0.02", g, 0) 
#agent = detection.HOLMES(g, 0, 1, strategy_aware, other_kinds, L)
# attacker = attackers.StackelbergAttacker(g, 1, 1)
attacker = attackers.StochasticAttacker(g, 1, 1, *distribution1)
g.set_players([agent], [attacker])

In [76]:
e1 = runner.Experiment(g)

In [77]:
e1.run_interaction()
print(game_str(e1.game, lenght=7))
print(e1.agent.belief)
print(e1.game.strategy_history[-1])

□	 	 	 	
5	4	8	1	
 	△	 	 	
{<StackelbergAttacker id:1 resources:1>: 0.0, <StochasticAttacker id:1 resources:1>: 1.0}
{0: [0.30934782608695643, 0.13543478260869565, 0.5552173913043479, 0.0], 1: [0.17, 0.8, 0.01, 0.02]}
